In [1]:
!git pull origin main

From https://github.com/jhallend/nfl_ml
 * branch            main       -> FETCH_HEAD
Already up to date.


In [2]:
!git commit NFL_ML.ipynb -m "committing just "

hint: Waiting for your editor to close the file... 7="~/SageMaker/nfl_ml/.git/COMMIT_EDITMSG" 15L, 391C# Please enter the commit message for your changes. Lines starting
# with '#' will be ignored, and an empty message aborts the commit.
#
# Committer: EC2 Default User <ec2-user@ip-172-16-49-156.us-east-2.compute.interrnal>
#
# On branch main
# Your branch is up to date with 'origin/main'.
#
# Changes to be committed:
#modified:   NFL_ML.ipynb
#
# Untracked files:
#.ipynb_checkpoints/
#
~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                                                                               ~                         

In [11]:
!git push origin

/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509
/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p27/lib/python2.7/site-packages/OpenSSL/crypto.py:12: CryptographyDeprecationWarning: Python 2 is no longer supported by the Python core team. Support for it is now deprecated in cryptography, and will be removed in the next release.
  from cryptography import x509
Everything up-to-date


In [3]:
!pip install keras
!pip install tensorflow
!pip install xgboost
!pip install imblearn
!pip install cvxpy

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2

In [4]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
from keras.regularizers import l1,l2
from keras.models import model_from_json
from keras.models import clone_model
from keras.layers import Dropout
from xgboost import XGBClassifier

from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from imblearn.over_sampling import SMOTE
import pandas as pd
import numpy as np
import cvxpy
import matplotlib.pyplot as plt
import time
import re
import sqlalchemy
import psycopg2
import requests
import lxml.html as lh
from datetime import datetime
from datetime import timedelta
import calendar
from bs4 import BeautifulSoup
import queue
import threading
import sys
import io
from io import StringIO
import re
import math
import os
import glob

ImportError: No module named imblearn.over_sampling

In [ ]:
import boto3, re, sys, math, json, os, sagemaker, urllib.request
from sagemaker import get_execution_role
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Image
from IPython.display import display
from time import gmtime, strftime
from sagemaker.predictor import csv_serializer

In [ ]:
pd.options.display.max_columns = None

In [ ]:
# Define IAM role
role = get_execution_role()
prefix = 'sagemaker/DEMO-xgboost-dm'
my_region = boto3.session.Session().region_name # set the region of the instance

In [ ]:
# bucket_name = 'nfl-ml-pff-data' # <--- CHANGE THIS VARIABLE TO A UNIQUE NAME FOR YOUR BUCKET
# s3 = boto3.resource('s3')
# try:
#     if  my_region == 'us-east-1':
#         s3.create_bucket(Bucket=bucket_name)
#     else: 
#         s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={ 'LocationConstraint': my_region })
#     print('S3 bucket created successfully')
# except Exception as e:
#     print('S3 error: ',e)

In [ ]:
bucket='nfl-ml-pff-data'
file_key = 'dftrain_aws.csv'
s3uri = 's3://{}/{}'.format(bucket, file_key)
df = pd.read_csv(s3uri)

In [ ]:
df.drop('Unnamed: 0', inplace=True, axis=1)

In [ ]:
df = df.fillna(0)
df.reset_index(drop=True,inplace=True)

In [ ]:
df.drop_duplicates(subset=['player','team_name','season','week'], inplace=True)

In [ ]:
df = df.query('weekref!=5 or season !=2021')

In [ ]:
df.query('weekref!=5 or season !=2021')[[p for p in df.columns if (p=='week' or p=='season' or p=='team_name' or 'player' in p or 'fpts' in p) and 'id' not in p and 'count' not in p]]


In [ ]:
#SAVE THIS - VERY USEFUL QUERY
#This returns all rows where any player column is null, and it shows week, season, and player names
df[df[[y for y in df.columns if 'player' in y]].isna().any(axis=1)]\
[[p for p in df.columns if (p=='week' or p=='season' or p=='team_name' 
                                 or 'player' in p or 'fpts' in p) and 'def' not in p
                                  and 'id' not in p and 'count' not in p]]

In [ ]:
numericXCols = [c for c in df.columns if not any(ext in c for ext in ['week','weekly_rank','fpts','fd_points','dk_points',
                                                                      'single_game','player','player_id','position',
                                                                      'team','name','season','rank_join_helper_id','oppt','starter'])]


In [ ]:
xAll = df[numericXCols]
xAll.fillna(0,inplace=True)

In [ ]:
xAll.reset_index(drop=True,inplace=True)

In [ ]:
df[df[[y for y in df.columns if 'player' in y]].isna().any(axis=1)][[p for p in df.columns if (p=='week' or p=='season' or p=='team_name' or 'player' in p or 'points' in p) and 'id' not in p and 'count' not in p]]


In [ ]:
bitBit = False
past = time.time()
scQb = StandardScaler()
scWr1 = StandardScaler()
scWr2 = StandardScaler()
scWr3 = StandardScaler()
scRb1 = StandardScaler()
scRb2 = StandardScaler()
scTe = StandardScaler()
#QB
print('Start parsing for QB')
#dfNonZeroWr1=df[df["dk_points_rec1"]!=0.0]
#xWr1=dfNonZeroWr1[numericXCols]
basePoints = 'dk_points'
yQb=df['dk_points']
xQbTrain, xQbTest, yQbTrain, yQbTest = train_test_split(xAll, yQb, test_size=0.2, random_state=101)
print('Start 2nd half of QB after ' + str(time.time()-past))
if bitBit:
    yQbBitTrain = yQbTrain>20
    yQbBitTest = yQbTest>20
else:
    yQbBitTrain = yQbTrain
    yQbBitTest = yQbTest
scQb.fit(xQbTrain.astype(np.float))
xQbStdTrain = scQb.transform(xQbTrain.astype(np.float))
xQbStdTest = scQb.transform(xQbTest.astype(np.float))
if bitBit:
    xQbTrainSmote, yQbTrainSmote = SMOTE.fit_resample(xQbStdTrain, yQbBitTrain)

#WR1
print('Start parsing for WR1')
#dfNonZeroWr1=df[df["dk_points_rec1"]!=0.0]
#xWr1=dfNonZeroWr1[numericXCols]
yWr1=df[basePoints+'_rec1']
xWr1Train, xWr1Test, yWr1Train, yWr1Test = train_test_split(xAll, yWr1, test_size=0.2, random_state=101)
print('Start 2nd half of WR1 after ' + str(time.time()-past))
if bitBit:
    yWr1BitTrain = yWr1Train>20
    yWr1BitTest = yWr1Test>20
else:
    yWr1BitTrain = yWr1Train
    yWr1BitTest = yWr1Test
scWr1.fit(xWr1Train.astype(np.float))
xWr1StdTrain = scWr1.transform(xWr1Train.astype(np.float))
xWr1StdTest = scWr1.transform(xWr1Test.astype(np.float))
if bitBit:
    xWr1TrainSmote, yWr1TrainSmote = SMOTE.fit_resample(xWr1StdTrain, yWr1BitTrain)

#WR2
print('Start parsing for WR2 after ' + str(time.time()-past))
#dfNonZeroWr2=df[df["dk_points_rec2"]!=0.0]
#xWr2=dfNonZeroWr2[numericXCols]
yWr2=df[basePoints+'_rec2']
xWr2Train, xWr2Test, yWr2Train, yWr2Test = train_test_split(xAll, yWr2, test_size=0.2, random_state=101)
xWr2Train.fillna(0,inplace=True)
print('Start 2nd half of WR2 after ' + str(time.time()-past))
if bitBit:
    yWr2BitTrain = yWr2Train>20
    yWr2BitTest = yWr2Test>20
else:
    yWr2BitTrain = yWr2Train
    yWr2BitTest = yWr2Test
scWr2.fit(xWr2Train.astype(np.float))
xWr2StdTrain = scWr2.transform(xWr2Train.astype(np.float))
xWr2StdTest = scWr2.transform(xWr2Test.astype(np.float))
if bitBit:
    xWr2TrainSmote, yWr2TrainSmote = SMOTE.fit_resample(xWr2StdTrain, yWr2BitTrain)

#WR3
print('Start parsing for WR3 after ' + str(time.time()-past))
#dfNonZeroWr3=df[df["dk_points_rec3"]!=0.0]
#xWr3=dfNonZeroWr3[numericXCols]
yWr3=df[basePoints+'_rec3']
xWr3Train, xWr3Test, yWr3Train, yWr3Test = train_test_split(xAll, yWr3, test_size=0.2, random_state=101)
print('Start 2nd half of WR3 after ' + str(time.time()-past))
if bitBit:
    yWr3BitTrain = yWr3Train>20
    yWr3BitTest = yWr3Test>20
else:
    yWr3BitTrain = yWr3Train
    yWr3BitTest = yWr3Test
scWr3.fit(xWr3Train.astype(np.float))
xWr3StdTrain = scWr3.transform(xWr3Train.astype(np.float))
xWr3StdTest = scWr3.transform(xWr3Test.astype(np.float))
if bitBit:
    xWr3TrainSmote, yWr3TrainSmote = SMOTE.fit_resample(xWr3StdTrain, yWr3BitTrain)

#RB1
print('Start parsing for RB1 after ' + str(time.time()-past))
#dfNonZeroRb1=df[df["dk_points_rus1"]!=0.0]
#xRb1=dfNonZeroRb1[numericXCols]
yRb1=df[basePoints+'_rus1']
xRb1Train, xRb1Test, yRb1Train, yRb1Test = train_test_split(xAll, yRb1, test_size=0.2, random_state=101)
print('Start 2nd half of RB1 after ' + str(time.time()-past))
if bitBit:
    yRb1BitTrain = yRb1Train>20
    yRb1BitTest = yRb1Test>20
else:
    yRb1BitTrain = yRb1Train
    yRb1BitTest = yRb1Test
scRb1.fit(xRb1Train.astype(np.float))
xRb1StdTrain = scRb1.transform(xRb1Train.astype(np.float))
xRb1StdTest = scRb1.transform(xRb1Test.astype(np.float))
if bitBit:
    xRb1TrainSmote, yRb1TrainSmote = SMOTE.fit_resample(xRb1StdTrain, yRb1BitTrain)

#RB2
print('Start parsing for RB2 after ' + str(time.time()-past))
#dfNonZeroRb2=df[df["dk_points_rus2"]!=0.0]
#xRb2=dfNonZeroRb2[numericXCols]
yRb2=df[basePoints+'_rus2']
xRb2Train, xRb2Test, yRb2Train, yRb2Test = train_test_split(xAll, yRb2, test_size=0.2, random_state=101)
print('Start 2nd half of RB2 after ' + str(time.time()-past))
if bitBit:
    yRb2BitTrain = yRb2Train>20
    yRb2BitTest = yRb2Test>20
else:
    yRb2BitTrain = yRb2Train
    yRb2BitTest = yRb2Test
scRb2.fit(xRb2Train.astype(np.float))
xRb2StdTrain = scRb2.transform(xRb2Train.astype(np.float))
xRb2StdTest = scRb2.transform(xRb2Test.astype(np.float))
if bitBit:
    xRb2TrainSmote, yRb2TrainSmote = SMOTE.fit_resample(xRb2StdTrain, yRb2BitTrain)
print('Finished after ' + str(time.time()-past))

#TE
print('Start parsing for TE after ' + str(time.time()-past))
#dfNonZeroRb2=df[df["dk_points_rus2"]!=0.0]
#xRb2=dfNonZeroRb2[numericXCols]
yTe=df[basePoints+'_te']
xTeTrain, xTeTest, yTeTrain, yTeTest = train_test_split(xAll, yTe, test_size=0.2, random_state=101)
print('Start 2nd half of TE after ' + str(time.time()-past))
if bitBit:
    yTeBitTrain = yTeTrain>20
    yTeBitTest = yTeTest>20
else:
    yTeBitTrain = yTeTrain
    yTeBitTest = yTeTest
scTe.fit(xTeTrain.astype(np.float))
xTeStdTrain = scTe.transform(xTeTrain.astype(np.float))
xTeStdTest = scTe.transform(xTeTest.astype(np.float))
if bitBit:
    xTeTrainSmote, yTeTrainSmote = SMOTE.fit_resample(xTeStdTrain, yTeBitTrain)
print('Finished after ' + str(time.time()-past))

In [ ]:
classifier = Sequential()
classifier.add(Dense(activation = 'relu', 
                          input_dim = len(xAll.columns), 
                          units = 5000,#len(xYardsTrain.columns), 
                          kernel_initializer = 'uniform', 
                          #W_regularizer=l2(0.001), this is the same as kernel_regularizer, and Keras wants you to use this
                          kernel_regularizer=l2(0.001)))
# Adding the second hidden layer
classifier.add(Dense(activation = 'relu', units = 10, kernel_initializer = 'uniform', kernel_regularizer='l1'))
#classifierYards.add(Dropout(0.1))
#classifier.add(Dense(1,activation='sigmoid'))
classifier.add(Dense(1))
#https://keras.io/api/losses/regression_losses/#meansquarederror-class
opt_adam = tf.keras.optimizers.Adam(clipnorm=1.0)
opt_adam = tf.keras.optimizers.Adam(clipvalue=0.5)
classifier.compile(optimizer = opt_adam, loss = 'mean_squared_error')#, metrics = ['accuracy'])

In [ ]:
modelDict={}

In [ ]:
classifierQb=clone_model(classifier)
classifierQb.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierWr1=clone_model(classifier)
classifierWr1.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierWr2=clone_model(classifier)
classifierWr2.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierWr3=clone_model(classifier)
classifierWr3.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierRb1=clone_model(classifier)
classifierRb1.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierRb2=clone_model(classifier)
classifierRb2.compile(optimizer = opt_adam, loss = 'mean_squared_error')
classifierTe=clone_model(classifier)
classifierTe.compile(optimizer = opt_adam, loss = 'mean_squared_error')

In [ ]:
client = boto3.client('s3')
client.download_file(bucket,
                     'classifierQb_20211010.h5',
                     'classifierQb_20211010.h5')

In [ ]:
classifierQb.load_weights('classifierQb_20211117.h5')

In [ ]:
# weights_file_key = 'classifierYards_20211005.h5'
# s3WeightsUri = 's3://{}/{}'.format(bucket, weights_file_key)

# classifierQb.load_weights(s3WeightsUri)

In [ ]:
modelDict[1]=classifierQb
modelDict[3]=classifier3
modelDict[4]=classifier4
modelDict[5]=classifier5
modelDict[6]=classifier6
modelDict[7]=classifier7
modelDict[8]=classifier8
modelDict[9]=classifier9
modelDict[10]=classifier10